In [1]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors
sns.set_theme(style="whitegrid")#, palette="colorblind")
tol_colors = [
    "#88CCEE", "#332288", "#44AA99", "#117733", "#999933",
    "#DDCC77", "#CC6677", "#882255", "#AA4499", "#DDDDDD"]
alpha_value = 0.8  # Adjust transparency here
tol_colors = [(mcolors.to_rgba(c, alpha=alpha_value)) for c in tol_colors]
#sns.set_palette(tol_colors)
import numpy as np
import ast
from scipy.stats import skewtest, mannwhitneyu, fisher_exact, wilcoxon
import numpy as np
from scipy.stats import kendalltau
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='white')

In [2]:
turn_labels = {}
eli5_df = pd.read_pickle("eli5_ds/annotation-results/MACE-measure/final_mace_predictions.pkl")

eli5_df['exp_act_label'] = eli5_df.exp_act_label.apply(lambda x: '(E10) Other' if x == '(E09) Other' else x)
eli5_df['exp_act_label'] = eli5_df.exp_act_label.apply(lambda x: '(E09) Introducing Extraneous Information' if x == '(E10) Introducing Extraneous Information' else x)

eli5_df['dlg_act_label'] = eli5_df.dlg_act_label.apply(lambda x: '(D09) Other' if x == '(D10) Other' else x)
eli5_df['dlg_act_label'] = eli5_df.dlg_act_label.apply(lambda x: '(D10) To provide informing statement' if x == '(D09) To provide informing statement' else x)

eli5_df['dlg_act_label'] = eli5_df.dlg_act_label.apply(lambda x: '(D06) Answer - Other' if x == '(D06) To answer - Other' else x)
eli5_df['dlg_act_label'] = eli5_df.dlg_act_label.apply(lambda x: '(D07) Agreement' if x == '(D07) To provide agreement statement' else x)
eli5_df['dlg_act_label'] = eli5_df.dlg_act_label.apply(lambda x: '(D08) Disagreement' if x == '(D08) To provide disagreement statement' else x)

turn_labels["dlg_act_label"] = sorted(list(eli5_df.dlg_act_label.unique()))
turn_labels["exp_act_label"] = sorted(list(eli5_df.exp_act_label.unique()))
#turn_labels["topic_func_label"] = sorted(list(eli5_df.topic_func_label.unique()))
turn_labels

{'dlg_act_label': ['(D01) To ask a check question',
  '(D02) To ask what/how question',
  '(D03) To ask other kind of questions',
  '(D04) To answer a question by confirming',
  '(D05) To answer a question by disconfirming',
  '(D06) Answer - Other',
  '(D07) Agreement',
  '(D08) Disagreement',
  '(D09) Other',
  '(D10) To provide informing statement'],
 'exp_act_label': ['(E01) Testing understanding',
  '(E02) Testing prior knowledge',
  '(E03) Provide an explanation',
  '(E04) Ask for an explanation',
  '(E05) Signaling understanding',
  '(E06) Signaling non-understanding',
  '(E07) Providing Feedback',
  '(E08) Providing Assessment',
  '(E09) Introducing Extraneous Information',
  '(E10) Other']}

In [3]:
df = pd.read_pickle("final_mace_predictions_longformer-base-4096.pkl")

In [4]:
df.head()

,task_id,turn_id,topic,system_prompt,turn_text,turn_text_with_topic,dlg_act_label_predictions,exp_act_label_predictions,topic_func_label_predictions
0,31479367-972d-4be0-a5b7-b545b8667bd1,0,The board game Quarto and its rules,base,"{'author': 'Explanee', 'text': 'Hello'}","{'author': 'Explanee', 'text': 'The board game...",(D09) Other,(E10) Other,(T04) Other - No topic was introduced
1,31479367-972d-4be0-a5b7-b545b8667bd1,1,The board game Quarto and its rules,base,"{'author': 'Explainer', 'text': 'Hello. Welcom...","{'author': 'Explainer', 'text': 'The board gam...",(D02) To ask what/how question,(E02) Testing prior knowledge,(T04) Other - No topic was introduced
2,31479367-972d-4be0-a5b7-b545b8667bd1,2,The board game Quarto and its rules,base,"{'author': 'Explanee', 'text': 'I would like t...","{'author': 'Explanee', 'text': 'The board game...",(D06) Answer - Other,(E03) Provide an explanation,(T01) It is the main topic
3,31479367-972d-4be0-a5b7-b545b8667bd1,3,The board game Quarto and its rules,base,"{'author': 'Explainer', 'text': 'Quatro is a s...","{'author': 'Explainer', 'text': 'The board gam...",(D06) Answer - Other,(E03) Provide an explanation,(T01) It is the main topic
4,31479367-972d-4be0-a5b7-b545b8667bd1,4,The board game Quarto and its rules,base,"{'author': 'Explanee', 'text': 'Explain all th...","{'author': 'Explanee', 'text': 'The board game...",(D06) Answer - Other,(E03) Provide an explanation,(T01) It is the main topic


In [10]:
df = pd.read_pickle("final_mace_predictions_longformer-base-4096.pkl")
turn_counts = {}
for author in ["Explanee", "Explainer"]:
    turn_counts[author] = {}
    for system_prompt in ["base", "enhanced"]:
        turn_counts[author][system_prompt] = {}
        for turn_label in turn_labels:
            
            turn_counts[author][system_prompt][turn_label] = {}
            df.apply(lambda row: row['turn_text']['author'], axis=1)
            # Filter dataframe
            filtered_df = df[
                (df["system_prompt"] == system_prompt) &
                (df["turn_text"].map(lambda x: x["author"] == author))
            ]
            
            grouped_counts = (
                filtered_df.groupby("task_id")[turn_label + "_predictions"]
                .value_counts()
                .unstack(fill_value=0)  # Ensure all categories appear, filling missing ones with zero
                .stack()
            )
            
            num_queries = grouped_counts.groupby('task_id').sum(numeric_only=True).to_dict()
            
            grouped_counts = grouped_counts.to_dict()
            
            for (task_id, turn), value in grouped_counts.items():
                if turn not in turn_counts[author][system_prompt][turn_label]:
                    turn_counts[author][system_prompt][turn_label][turn] = []
                turn_counts[author][system_prompt][turn_label][turn].append(value/num_queries[task_id])

p_values = {}
for author in ["Explanee", "Explainer"]:
    p_values[author] = {}
    for turn_label in turn_labels:
        p_values[author][turn_label] = {}
        for turn in turn_labels[turn_label]:
            if turn in turn_counts[author]["base"][turn_label] and turn in turn_counts[author]["enhanced"][turn_label]:
                data_base = turn_counts[author]["base"][turn_label][turn]
                data_enhanced = turn_counts[author]["enhanced"][turn_label][turn]
                statistic, p_value = mannwhitneyu(data_base, data_enhanced)
                p_values[author][turn_label][turn] = p_value

with open("p_values_turn_counts.json", "w") as file:
    json.dump(p_values, file, indent=4)
                
for author in ["Explanee", "Explainer"]:
    for system_prompt in ["base", "enhanced"]:
        for turn_label in turn_labels:
            for turn in turn_labels[turn_label]:
                if turn not in turn_counts[author][system_prompt][turn_label]:
                    turn_counts[author][system_prompt][turn_label][turn] = 0
                else:
                    values_mean = np.mean(turn_counts[author][system_prompt][turn_label][turn])
                    values_std = np.std(turn_counts[author][system_prompt][turn_label][turn])
                    turn_counts[author][system_prompt][turn_label][turn] = values_mean

for author in ["Explanee", "Explainer"]:
    data = []

    for turn_label in turn_labels:
        rest_values = {"base": 0, "enhanced": 0}
        
        values_base_sum = sum(list(turn_counts[author]["base"][turn_label].values()))
        values_enhanced_sum = sum(list(turn_counts[author]["enhanced"][turn_label].values()))

        for turn in turn_counts[author]["base"][turn_label]:
            value_base = turn_counts[author]["base"][turn_label][turn] / values_base_sum
            value_enhanced = turn_counts[author]["enhanced"][turn_label][turn] / values_enhanced_sum
            if author == "Explanee":
                threshold = 0.05
            else:
                threshold = 0.02
            if (value_base < threshold and value_enhanced < threshold) or "Other" in turn:
                rest_values["base"] += value_base
                rest_values["enhanced"] += value_enhanced
            else:
                data.append({
                    "Category": turn, 
                    "Proportion": value_base, 
                    "Type": "base", 
                    "Label": turn_label
                })
                data.append({
                    "Category": turn, 
                    "Proportion": value_enhanced, 
                    "Type": "enhanced", 
                    "Label": turn_label
                })

        for system_prompt in rest_values:
            data.append({
                "Category": "Rest", 
                "Proportion": rest_values[system_prompt], 
                "Type": system_prompt, 
                "Label": turn_label
            })

    df = pd.DataFrame(data)

    # Create subplots for each turn_label within the same figure
    fig, axes = plt.subplots(1, len(turn_labels), figsize=(7.5, 4.2), sharey=True)
    turn_label_dict = {"dlg_act_label": "Dialogue Acts", "exp_act_label": "Explanation Moves"}

    for i, turn_label in enumerate(turn_labels):
        ax = axes[i]
        df_subset = df[df["Label"] == turn_label].copy()
        df_pivot = df_subset.pivot(index="Type", columns="Category", values="Proportion")
        df_pivot = df_pivot.fillna(0)

        total_categories = df_pivot.columns

        df_pivot = df_pivot[df_pivot.columns[::-1]]
        
        df_pivot.plot(
            kind="bar", 
            stacked=True, 
            ax=ax, 
            color=tol_colors[-1:] + tol_colors[:len(total_categories)-1]
        )

        ax.set_xlabel("")
        ax.set_title(turn_label_dict[turn_label], fontsize=9)
        ax.set_ylim(0, 1.0)
        ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
        ax.yaxis.tick_right()
        ax.set_yticklabels(["0%", "10%", "20%", "30%", "40%", "50%", "60%", "70%", "80%", "90%", "100%"], fontsize=9)
        for label in ax.get_yticklabels():
            label.set_horizontalalignment('center')
        ax.tick_params(axis='y', length=0)
        ax.set_xticks([0, 1])
        ax.set_xticklabels(["Base", "Enhanced"], fontsize=9)
        ax.grid(False)
        ax.legend_.remove()
        ax.set_facecolor('none')

        for spine in ax.spines.values():
            spine.set_visible(False)

        ax.tick_params(axis='x', labelrotation=0)
            
        # Add labels inside bars
        for j, (setting, row) in enumerate(df_pivot.iterrows()):
            cumulative = 0
            for category in df_pivot.columns:
                value = row[category]
                if category in p_values[author][turn_label]:
                    p_value = p_values[author][turn_label][category]
                else:
                    p_value = 1
                if value >= 0.05:
                    label = f"{value*100:.1f}"
                    if p_value < 0.05:
                        label = r"$^\dagger$"+label
                    ax.text(
                        j, cumulative + value / 2, 
                        label, 
                        ha="center", va="center", fontsize=9, color="black"
                    )
                category = category.split(") ")[-1] 
                if i == 0 and setting == "base" :
                        ax.text(
                            j-0.3, cumulative + value/2, 
                            "  "+category, 
                            ha="right", va="center", fontsize=9, color="black"
                        )
                elif i == 1 and setting == "enhanced" :
                    ax.text(
                        j+0.3, cumulative + value/2, 
                        category+"  ", 
                        ha="left", va="center", fontsize=9, color="black"
                    )
                    
                cumulative += value
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.04)
    plt.savefig(f"turn_labels_{author}_combined.pdf", format="pdf")

In [9]:
with open("../../evaluation/results/results_per_user.json", "r") as file:
    stats = json.load(file)
with open("../../evaluation/user_study_data/setup_per_user.json", "r") as file:
    setup_per_user = json.load(file)

    
turn_counts = {}
p_values = {}
df = pd.read_pickle("final_mace_predictions_longformer-base-4096.pkl")

for author in ["Explanee", "Explainer"]:
    turn_counts[author] = {}
    turn_counts[author] = {}
    for turn_label in turn_labels:
        turn_counts[author][turn_label] = {}
        df.apply(lambda row: row['turn_text']['author'], axis=1)
        # Filter dataframe
        filtered_df = df[
            (df["system_prompt"] == "enhanced") &
            (df["turn_text"].map(lambda x: x["author"] == author))
        ]

        grouped_counts = (
            filtered_df.groupby("task_id")[turn_label + "_predictions"]
            .value_counts()
            .unstack(fill_value=0)  # Ensure all categories appear, filling missing ones with zero
            .stack()
            .to_dict()
        )

        turn_counts[author][turn_label]["obj_comprehension"] = []
        turn_counts[author][turn_label]["enabledness"] = []

        old_task_id = None

        for (task_id, turn), value in grouped_counts.items():
            if turn not in turn_counts[author][turn_label]:
                turn_counts[author][turn_label][turn] = []
            turn_counts[author][turn_label][turn].append(value)
            if not old_task_id or task_id !=old_task_id:
                turn_counts[author][turn_label]["obj_comprehension"].append(stats[task_id]["post_obj_comprehension"])
                turn_counts[author][turn_label]["enabledness"].append(stats[task_id]["post_enabledness"])
            old_task_id = task_id

SIGNIFICANCE_LEVEL = 0.05  # Set threshold for statistical significance
for author in ["Explanee", "Explainer"]:
    for turn_label in ["dlg_act_label", "exp_act_label"]:
        # Extract values dynamically
        data = np.column_stack([turn_counts[author][turn_label][metric] for metric in turn_counts[author][turn_label]])

        # Create DataFrame
        df = pd.DataFrame(data, columns=turn_counts[author][turn_label])

        # Compute Kendall correlation matrix
        corr_matrix = df.corr(method="kendall")

        # Compute p-values
        p_values = np.zeros(corr_matrix.shape)
        cols = df.columns

        for i in range(len(cols)):
            for j in range(len(cols)):
                if i != j:
                    tau, p_value = kendalltau(df[cols[i]], df[cols[j]])
                    p_values[i, j] = p_value
                else:
                    p_values[i, j] = np.nan  # No self-correlation p-value

        p_values_df = pd.DataFrame(p_values, columns=cols, index=cols)

        # Plot only significant correlations
        plt.figure(figsize=(12,7))
        ax = sns.heatmap(
            corr_matrix,  # Show full correlation matrix
            annot=True,
            fmt=".2f",
            linewidths=0.5,
            cmap="coolwarm",
            vmin=-1, vmax=1
        )

        # Bold only significant correlations
        for text, (i, j) in zip(ax.texts, [(row, col) for row in range(len(cols)) for col in range(len(cols))]):
            if i != j and p_values_df.iloc[i, j] < SIGNIFICANCE_LEVEL:
                text.set_weight("bold")

        plt.title(f"Significant Correlations ({author}, {turn_label})")
        plt.tight_layout()
        plt.savefig(f'{turn_label}_{author}_correlation_matrix.pdf', format="pdf")

### Text complexity

In [4]:
from textcomplexity import surface
from textcomplexity.utils.text import Text
from collections import namedtuple
import nltk
Token = namedtuple("Token", ["word", "pos"])
import spacy
nlp = spacy.load("en_core_web_sm")
import textstat

In [9]:
results = {}
for topic in ["Quarto", "black holes", "sleep"]:
    results[topic] = {}
    df = pd.read_pickle("final_mace_predictions_longformer-base-4096.pkl")
    filtered_df = df[(df["turn_text"].map(lambda x: x["author"] == "Explanee")) & (df["exp_act_label_predictions"]=="(E06) Signaling non-understanding")]

    gunning_fog = []
    type_token_ratio = []
    shannon_entropy = []

    for index, row in filtered_df.iterrows():
        turn_ids = range(row["turn_id"]-1, row["turn_id"] + 2)
        messages = df[
            (df["task_id"] == row["task_id"]) & 
            (df["turn_id"].isin(turn_ids))
        ]
        if len(messages) == len(turn_ids) and row["system_prompt"] == "enhanced" and topic in row["topic"] :
            # Check that the LLM provided an explanation before the user signaled non-understanding
            EX_provided_explanation = messages[messages['turn_id'] == row["turn_id"]-1]['exp_act_label_predictions'].iloc[0] == '(E03) Provide an explanation'
            if EX_provided_explanation:
                current_gunning_fog = []
                current_type_token_ratio = []
                current_shannon_entropy = []
                for index, message in messages.iterrows():
                    #print(message["turn_text"])
                    #print(message["exp_act_label_predictions"])
                    #print()
                    text_tokenized = nltk.word_tokenize(message["turn_text"]["text"])
                    text_tagged = nltk.pos_tag(text_tokenized)
                    tuple_list = [] 
                    for tuple_ in text_tagged:
                        tuple_list.append(Token(tuple_[0], tuple_[1]))
                    tc_text = Text.from_tokens(tuple_list)
                    gunning_fog_value = textstat.gunning_fog(message["turn_text"]["text"])
                    type_token_ratio_value = surface.type_token_ratio(tc_text)
                    shannon_entropy_value = surface.entropy(tc_text)
                    
                    if message["turn_text"]["author"] == "Explainer":
                        current_gunning_fog.append(gunning_fog_value)
                        current_type_token_ratio.append(type_token_ratio_value)
                        current_shannon_entropy.append(shannon_entropy_value)
                gunning_fog.append(current_gunning_fog)
                type_token_ratio.append(current_type_token_ratio)
                shannon_entropy.append(current_shannon_entropy)
                #print("----------------------------------------------------------------------------------")
                

    print(topic, len(gunning_fog))
    if len(gunning_fog) == 0:
        continue
    # Convert to NumPy array
    data_array = np.array(gunning_fog)
    # Calculate percentage changes
    percentage_changes = ((data_array[:, 1] - data_array[:, 0]) / data_array[:, 0]) * 100
    # Compute mean percentage change
    mean_percentage_change = np.mean(percentage_changes)
    std_percentage_change = np.std(percentage_changes)
    # Display result
    if mean_percentage_change > 0:
        print(f"Mean change in gunning fog index: +{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")
    else:
        print(f"Mean change in gunning fog index: -{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")

    # Convert to NumPy array
    data_array = np.array(type_token_ratio)
    # Calculate percentage changes
    percentage_changes = ((data_array[:, 1] - data_array[:, 0]) / data_array[:, 0]) * 100
    # Compute mean percentage change
    mean_percentage_change = np.mean(percentage_changes)
    std_percentage_change = np.std(percentage_changes)
    # Display result
    if mean_percentage_change > 0:
        print(f"Mean change in type-token-ratio: +{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")
    else:
        print(f"Mean change in type-token-ratio: -{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")

    # Convert to NumPy array
    data_array = np.array(shannon_entropy)
    # Calculate percentage changes
    percentage_changes = ((data_array[:, 1] - data_array[:, 0]) / data_array[:, 0]) * 100
    # Compute mean percentage change
    mean_percentage_change = np.mean(percentage_changes)
    std_percentage_change = np.std(percentage_changes)
    # Display result
    if mean_percentage_change > 0:
        print(f"Mean change in shannon entropy: +{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")
    else:
        print(f"Mean change in shannon entropy: -{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")

Quarto 2
Mean change in gunning fog index: +109.4\% $\pm$ 100.9
Mean change in type-token-ratio: +23.0\% $\pm$ 2.3
Mean change in shannon entropy: +7.1\% $\pm$ 8.9
black holes 8
Mean change in gunning fog index: --8.5\% $\pm$ 16.7
Mean change in type-token-ratio: +6.0\% $\pm$ 43.7
Mean change in shannon entropy: --1.6\% $\pm$ 5.3
sleep 4
Mean change in gunning fog index: --10.1\% $\pm$ 17.9
Mean change in type-token-ratio: +1.9\% $\pm$ 12.6
Mean change in shannon entropy: +0.8\% $\pm$ 7.1


In [25]:
results = {}
gunning_fog = []
type_token_ratio = []
shannon_entropy = []
    
for topic in ["Quarto", "black holes", "sleep"]:
    results[topic] = {}
    df = pd.read_pickle("final_mace_predictions_longformer-base-4096.pkl")
    filtered_df = df[(df["turn_text"].map(lambda x: x["author"] == "Explanee")) & (df["exp_act_label_predictions"]=="(E06) Signaling non-understanding")]

    for index, row in filtered_df.iterrows():
        turn_ids = range(row["turn_id"]-1, row["turn_id"] + 2)
        messages = df[
            (df["task_id"] == row["task_id"]) & 
            (df["turn_id"].isin(turn_ids))
        ]
        if len(messages) == len(turn_ids) and row["system_prompt"] == "enhanced" and topic in row["topic"] :
            # Check that the LLM provided an explanation before the user signaled non-understanding
            EX_provided_explanation = messages[messages['turn_id'] == row["turn_id"]-1]['exp_act_label_predictions'].iloc[0] == '(E03) Provide an explanation'
            if EX_provided_explanation:
                current_gunning_fog = []
                current_type_token_ratio = []
                current_shannon_entropy = []
                for index, message in messages.iterrows():
                    #print(message["turn_text"])
                    #print(message["exp_act_label_predictions"])
                    #print()
                    text_tokenized = nltk.word_tokenize(message["turn_text"]["text"])
                    text_tagged = nltk.pos_tag(text_tokenized)
                    tuple_list = [] 
                    for tuple_ in text_tagged:
                        tuple_list.append(Token(tuple_[0], tuple_[1]))
                    tc_text = Text.from_tokens(tuple_list)
                    gunning_fog_value = textstat.gunning_fog(message["turn_text"]["text"])
                    type_token_ratio_value = surface.type_token_ratio(tc_text)
                    shannon_entropy_value = surface.entropy(tc_text)
                    
                    if message["turn_text"]["author"] == "Explainer":
                        current_gunning_fog.append(gunning_fog_value)
                        current_type_token_ratio.append(type_token_ratio_value)
                        current_shannon_entropy.append(shannon_entropy_value)
                gunning_fog.append(current_gunning_fog)
                type_token_ratio.append(current_type_token_ratio)
                shannon_entropy.append(current_shannon_entropy)
                #print("----------------------------------------------------------------------------------")
                
    
print("all", len(gunning_fog))
# Convert to NumPy array
data_array = np.array(gunning_fog)
# Calculate percentage changes
percentage_changes = ((data_array[:, 1] - data_array[:, 0]) / data_array[:, 0]) * 100
# Compute mean percentage change
mean_percentage_change = np.mean(percentage_changes)
std_percentage_change = np.std(percentage_changes)
# Display result
if mean_percentage_change > 0:
    print(f"Mean change in gunning fog index: +{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")
else:
    print(f"Mean change in gunning fog index: -{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")

# Convert to NumPy array
data_array = np.array(type_token_ratio)
# Calculate percentage changes
percentage_changes = ((data_array[:, 1] - data_array[:, 0]) / data_array[:, 0]) * 100
# Compute mean percentage change
mean_percentage_change = np.mean(percentage_changes)
std_percentage_change = np.std(percentage_changes)
# Display result
if mean_percentage_change > 0:
    print(f"Mean change in type-token-ratio: +{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")
else:
    print(f"Mean change in type-token-ratio: -{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")

# Convert to NumPy array
data_array = np.array(shannon_entropy)
# Calculate percentage changes
percentage_changes = ((data_array[:, 1] - data_array[:, 0]) / data_array[:, 0]) * 100
# Compute mean percentage change
mean_percentage_change = np.mean(percentage_changes)
std_percentage_change = np.std(percentage_changes)
# Display result
if mean_percentage_change > 0:
    print(f"Mean change in shannon entropy: +{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")
else:
    print(f"Mean change in shannon entropy: -{mean_percentage_change:.1f}\\% $\\pm$ {std_percentage_change:.1f}")

all 14
Mean change in gunning fog index: +7.9\% $\pm$ 58.5
Mean change in type-token-ratio: +7.3\% $\pm$ 34.4
Mean change in shannon entropy: +0.3\% $\pm$ 7.1
